# Online anlaysis for loop experiments

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from ScanImageTiffReader import ScanImageTiffReader
from roipoly import MultiRoi
from scipy.ndimage import mean 
import scipy.signal as signal
import asyncio
import ipywidgets as widgets
import matplotlib.animation as anim


def AOIselector(imageIn):
    ### imagIn expected to be single frame###
    %matplotlib 
    plt.imshow(imageIn)
    plt.show()
    my_rois = MultiRoi(roi_names=['region to keep'])
    %matplotlib inline

    BulbMask = my_rois.rois['region to keep'].get_mask(imageIn)
#     BulbMask+=my_rois.rois['bulb2'].get_mask(imageIn)
    return BulbMask


## change directory to your newly created folder

In [2]:
%cd D:\ScanimageData\Test

D:\ScanimageData\Test


## 1st run a single trail to define the ROI
- then draw your roi in the pop up window

In [3]:
%matplotlib inline

global oldfiles
old_files = glob.glob('*.tif')
vol=ScanImageTiffReader(old_files[0]).data();
imMean = np.mean(vol,axis=0)
mask=AOIselector(imMean)
fig1, ax = plt.subplot_mosaic([
    ['image','trace'],]
    ,constrained_layout=True,figsize=(10,6))

ax['image'].imshow(imMean)
ax['image'].axis('off')
global roi
roi = np.ma.masked_where(mask == 0,mask)
ax['image'].imshow(roi,alpha=0.2,cmap='Reds')
trace = np.zeros(vol.shape[0])
for i in range(vol.shape[0]):
    trace[i]=mean(vol[i,:,:],labels=mask,)
trace = signal.savgol_filter(trace,17,3)
ax['trace'].plot(trace)

### then run the next cell to generate the figure which will start to update as files are acquired
- you can minimise this jupyter window but do not close it

In [6]:
%matplotlib 
fig, ax = plt.subplot_mosaic([
    ['image','trace'],
    ['TC','TC'],
],constrained_layout=True,figsize=(10,6))

ax['image'].imshow(imMean)
ax['image'].axis('off')
global roi
ax['image'].imshow(roi,alpha=0.2,cmap='Reds')

trace = np.zeros(vol.shape[0])
for i in range(vol.shape[0]):
    trace[i]=mean(vol[i,:,:],labels=mask,)
trace = signal.savgol_filter(trace,17,3)
ax['trace'].plot(trace)
global amp
amp = []
amp.append(np.max(trace))
ax['TC'].plot(amp,'o')


def animate(i):
    global amp
    global roi
    global old_files
    current_files=glob.glob('*.tif')
    if len(current_files) > len(old_files) and len(current_files)>3:
        vol=ScanImageTiffReader(current_files[-2]).data()
        curr_Mean = np.mean(vol,axis=0)
        curr_trace = np.zeros(vol.shape[0])
        for i in range(vol.shape[0]):
            curr_trace[i]=mean(vol[i,:,:],labels=mask,)
        curr_trace = signal.savgol_filter(curr_trace,17,3)
        amp.append(np.max(curr_trace))

        ax['image'].clear()
        ax['image'].imshow(curr_Mean)
        ax['image'].imshow(roi,alpha=0.2,cmap='Reds')
        ax['trace'].clear()
        ax['trace'].plot(trace)
        ax['trace'].plot(curr_trace)
        ax['TC'].clear()
        ax['TC'].plot(amp,'o')
        del vol
        old_files = current_files
          
ani = anim.FuncAnimation(fig,animate,interval=2000)
plt.show()

Using matplotlib backend: QtAgg
